In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/unprejudiced/

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [ ]:
# install as colab doesn't have transformers in default
!pip install transformers
!pip install colorama

In [ ]:
import sys
sys.path.append("./")
sys.path.append("./StereoSet/")
sys.path.append("./StereoSet/code")
sys.path.append("./StereoSet/data")
from StereoSet.code.eval_generative_models import BiasEvaluator
from StereoSet.code import evaluation
from model import tokenizer,get_model
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import os
import json

In [ ]:
# trained_model_out = model
# checkpoint = torch.load('trained_models/model_indim50_lr001_noln_outnet_bz100_gpt.pth')
# trained_model_out.load_state_dict(checkpoint['model_state_dict'])
# trained_model_out = trained_model_out.to("cuda")

In [ ]:
# evaluator = BiasEvaluator(trained_intra_model=trained_model_out, input_file='./StereoSet/data/dev.json')

In [ ]:
trained_model_out = get_model(gpt2_name='gpt2',in_net=False,in_net_init_identity=False,out_net=False,out_net_init_identity=False,freeze_ln = False,freeze_pos = True, \
                freeze_wte = True, freeze_ff = True, freeze_attn = True)

for name, p in trained_model_out.named_parameters():
  p.requires_grad = False

checkpoint = torch.load('trained_models/gpt2/ln_unfrozen/model_indim50_lr001_newlmhaed_nobias_bz100_gpt.pth')
trained_model_out.load_state_dict(checkpoint['model_state_dict'])
trained_model_out = trained_model_out.to("cuda")

evaluator = BiasEvaluator(trained_intra_model=trained_model_out, input_file='./StereoSet/data/dev.json')
intrasentence_bias = evaluator.evaluate_intrasentence()

bias = {}
bias['intrasentence'] = intrasentence_bias
with open("predictions/gpt2/ln_unfrozen/model_indim50_lr001_newlmhaed_nobias_bz100_gpt.json", "w+") as f:
  json.dump(bias,f, indent=2)

In [ ]:
trained_model_out = get_model(gpt2_name='gpt2',in_net=False,in_net_init_identity=False,out_net=True,out_net_init_identity=True,freeze_ln = False,freeze_pos = True, \
                freeze_wte = True, freeze_ff = True, freeze_attn = True)

for name, p in trained_model_out.named_parameters():
  p.requires_grad = False

checkpoint = torch.load('trained_models/gpt2/ln_unfrozen/model_indim50_lr001_ln_outnet_bz100_gpt.pth')
trained_model_out.load_state_dict(checkpoint['model_state_dict'])
trained_model_out = trained_model_out.to("cuda")

evaluator = BiasEvaluator(trained_intra_model=trained_model_out, input_file='./StereoSet/data/dev.json')
intrasentence_bias = evaluator.evaluate_intrasentence()

bias = {}
bias['intrasentence'] = intrasentence_bias
with open("predictions/gpt2/ln_unfrozen/model_indim50_lr001_ln_outnet_bz100_gpt.json", "w+") as f:
  json.dump(bias,f, indent=2)

In [ ]:
trained_model_out = get_model(gpt2_name='gpt2',in_net=True,in_net_init_identity=True,out_net=True,out_net_init_identity=True,freeze_ln = True,freeze_pos = True, \
                freeze_wte = True, freeze_ff = True, freeze_attn = True)

for name, p in trained_model_out.named_parameters():
  p.requires_grad = False

checkpoint = torch.load('trained_models/gpt2/ln_frozen/model_indim50_lr001_inoutnet_bz100_gpt.pth')
trained_model_out.load_state_dict(checkpoint['model_state_dict'])
trained_model_out = trained_model_out.to("cuda")

evaluator = BiasEvaluator(trained_intra_model=trained_model_out, input_file='./StereoSet/data/dev.json')
intrasentence_bias = evaluator.evaluate_intrasentence()

bias = {}
bias['intrasentence'] = intrasentence_bias
with open("predictions/gpt2/ln_frozen/model_indim50_lr001_inoutnet_bz100_gpt.json", "w+") as f:
  json.dump(bias,f, indent=2)

In [ ]:
trained_model_out = get_model(gpt2_name='gpt2',in_net=True,in_net_init_identity=True,out_net=False,out_net_init_identity=False,freeze_ln = True,freeze_pos = True, \
                freeze_wte = True, freeze_ff = True, freeze_attn = True)

for name, p in trained_model_out.named_parameters():
  p.requires_grad = False

checkpoint = torch.load('trained_models/gpt2/ln_frozen/model_indim50_lr001_innet_bz100_gpt.pth')
trained_model_out.load_state_dict(checkpoint['model_state_dict'])
trained_model_out = trained_model_out.to("cuda")

evaluator = BiasEvaluator(trained_intra_model=trained_model_out, input_file='./StereoSet/data/dev.json')
intrasentence_bias = evaluator.evaluate_intrasentence()

bias = {}
bias['intrasentence'] = intrasentence_bias
with open("predictions/gpt2/ln_frozen/model_indim50_lr001_innet_bz100_gpt.json", "w+") as f:
  json.dump(bias,f, indent=2)

In [ ]:
# intrasentence_bias = evaluator.evaluate_intrasentence()

In [ ]:
# bias = {}
# bias['intrasentence'] = intrasentence_bias
# with open("predictions/model_indim50_lr001_noln_outnet_bz100_gpt.json", "w+") as f:
#   json.dump(bias,f, indent=2)

In [ ]:
!python ./StereoSet/code/evaluation.py \
    --gold-file './StereoSet/data/dev.json' \
    --predictions-file "./StereoSet/code/predictions/predictions_SentimentModel.json" \
    --output-file "stereo_eval_results/predictions_SentimentModel.json"

In [ ]:
!python ./StereoSet/code/evaluation.py \
    --gold-file './StereoSet/data/dev.json' \
    --predictions-file "./StereoSet/code/predictions/predictions_gpt2-medium_ModelNSP_GPT2LM.json" \
    --output-file "stereo_eval_results/predictions_gpt2-medium_ModelNSP_GPT2LM.json"

In [ ]:
!python ./StereoSet/code/evaluation.py \
    --gold-file './StereoSet/data/dev.json' \
    --predictions-file "predictions/gpt2/ln_unfrozen/model_indim50_lr001_newlmhaed_bz100_gpt.json" \
    --output-file "stereo_eval_results/gpt2/ln_unfrozen/model_indim50_lr001_newlmhaed_bz100_gpt.json"